In [5]:
'''
;====================================================================================================
;                              X2ABUNDANCE
;
; Package for determining elemental weight percentages from XRF line fluxes
;
; Algorithm developed by P. S. Athiray (Athiray et al. 2015)
; Codes in IDL written by Netra S Pillai
; Codes for XSPEC localmodel developed by Ashish Jacob Sam and Netra S Pillai
;
; Developed at Space Astronomy Group, U.R.Rao Satellite Centre, Indian Space Research Organisation
;
;====================================================================================================

This file contains the common functions/methods and class definitions used in the repository

The major library used for this repository is the open-source xraylib at https://github.com/tschoonj/xraylib which is available to be added as dependacy only via conda (Anaconda/miniconda). Thus it is essential to use conda virtual enviornment to execute the code.

The repository uses the following dependacies
	xraylib:    	installed by running conda install xraylib
	numpy: 		installed by running conda install numpy
	astropy: 	installed by running conda install astropy
'''
from typing import Any
import numpy as np

def n_elements(array)->int:
	if(type(array) is list):
		return array.__len__()
	else:#numpy array
		s = np.size(array)
		return s

def dblarr(*args:int) ->Any:
	return np.zeros(tuple(args))

def total(MyList:list) -> Any:
	total = 0
	if(type(MyList) is list):
		for i in MyList:
			total = total + i
	else:
		for i in np.nditer(MyList):
			total = total + i
	return total

def ChangeEveryElement(function,array:list) -> None:
	for i in range(0,array.__len__()):
		array[i]=function(array[i])

def readcol(filename:str, format:str=None)->tuple:
	if(format==None):
		rowformat=[]
	else: 
		rowformat=format.split(sep=',')
	TupleOfLists=[]
	for i in range(0,rowformat.__len__()):
		rowformat[i] = rowformat[i].capitalize()
		TupleOfLists.append([])
	import glob
	for filenames in glob.glob(filename):
		with open(filenames, 'r') as f:
			for line in f:
				inputstring=line.split(sep=None,maxsplit=-1)
				if(inputstring.__len__()==rowformat.__len__() and inputstring[0]):
					try:
						for i in range(0,inputstring.__len__()):
							if(i==TupleOfLists.__len__()):
								TupleOfLists.append([])
							if(rowformat[i]=='B' or rowformat[i]=='I' or rowformat[i]=='L' or rowformat[i]=='Z'):
								TupleOfLists[i].append(int(inputstring[i]))
							elif(rowformat[i]=='D' or rowformat[i]=='F'):
								TupleOfLists[i].append(float(inputstring[i]))
							elif(rowformat[i]!='X'):
								TupleOfLists[i].append(inputstring[i])
					except:
						continue

	TupleOfNpArray=[]
	for listitem in TupleOfLists:
		TupleOfNpArray.append(np.array(listitem))
	return tuple(TupleOfNpArray)


def SortVectors(TupleOfArrays:tuple, Reverse:bool = False) -> tuple:
	TupleOfLists = []
	for item in TupleOfArrays:
		myList = list(item)
		TupleOfLists.append(myList)
	ListOfTuples = []
	tuplelength = TupleOfLists.__len__()
	length = TupleOfLists[0].__len__()
	for w in TupleOfLists:
		if(w.__len__() != length):
			return ListOfTuples
	for i in range(0,length):
		tupleItem = []
		for k in TupleOfLists:
			tupleItem.append(k[i])
		ListOfTuples.append(tuple(tupleItem)) 
	ListOfTuples.sort(reverse=Reverse)
	for anylist in TupleOfLists:
		anylist.clear()
	for item in ListOfTuples:
		for i in range(0,tuplelength):
			TupleOfLists[i].append(item[i])
	TupleOfNpArray=[]
	for listitem in TupleOfLists:
		TupleOfNpArray.append(np.array(listitem))
	return tuple(TupleOfNpArray)


def totalLambda(function, array:list)->float:
	sum=0.0
	if(type(array) is list):
		for item in array:
			sum=sum+function(item)
	else:
		for item in np.nditer(array):
			sum=sum+function(item)
	return sum

def PRODUCT(array:list)->float:
	p=1
	if(type(array) is list):
		for item in array:
			p=p*item
	else:
		for item in np.nditer(array):
			p=p*item
	return p

def Tuple2String(MyRow:tuple)->str:
	output=""
	for x in MyRow:
		output=output+x.__str__()+" "
	return output

def file_lines(filename:str)->int:
	return sum(1 for _ in open(filename))

def fix(stuff):
	typeof = type(stuff)
	if(typeof is float):
		return int(stuff)
	elif typeof is list:
		intlist = []
		for f in stuff:
			intlist.append(int(f))
		return intlist
	else:#numpy array
		return stuff.astype(int)


def array_indices_custom(NpArray, *location)->list:
	ReturnValue = []
	if (type(location[0]) is list):
		location=list(location[0])
	else:
		location=list(location)
	for CurrentLocation in location:
		ArrayShape = list(np.shape(NpArray))
		prod = PRODUCT(ArrayShape)
		for i in range(0,ArrayShape.__len__()):
			item=ArrayShape[i]
			prod=prod/item
			ArrayShape[i]=int((CurrentLocation//prod)%item)
		ReturnValue.append(np.array(ArrayShape))
	return np.array(ReturnValue)

def strarr(count:int)->list:
	return ['']*count

class Xrf_Lines:
	def __init__(self, Edgeenergy, FluorYield, jumpfactor, RadRate, LineEnergy,Energy_nist, photoncs_nist, totalcs_nist,elename_string)  -> None:
		self.fluoryield = FluorYield
		self.edgeenergy = Edgeenergy
		self.jumpfactor = jumpfactor
		self.radrate = RadRate
		self.lineenergy = LineEnergy
		self.energy_nist=Energy_nist
		self.photoncs_nist = photoncs_nist
		self.totalcs_nist = totalcs_nist
		self.elename_string = elename_string


class Const_Xrf:
	def __init__(self,Musampletotal_echarline,Musampletotal_eincident,Muelementphoto_eincident) -> None:
		self.musampletotal_echarline=Musampletotal_echarline
		self.musampletotal_eincident = Musampletotal_eincident
		self.muelementphoto_eincident=Muelementphoto_eincident

class Xrf_Struc:
	def __init__(self,primary,secondary,total) -> None:
		self.primary_xrf=primary
		self.secondary_xrf=secondary
		self.total_xrf=total

class Scat_struc:
	def __init__(self,Coh,Incoh,Total) -> None:
		self.i_total=Total
		self.i_coh=Coh
		self.i_incoh=Incoh

class Spectrum:
	def __init__(self,energy,counts,xrf,scat_coh,scat_incoh,scat_total,xrf_lines_flux,xrf_lines_energy,pxrf,sxrf) -> None:
		self.energy=energy
		self.counts=counts
		self.xrf=xrf
		self.scat_coh=scat_coh
		self.scat_incoh=scat_incoh
		self.scat_total=scat_total
		self.xrf_lines_flux=xrf_lines_flux
		self.xrf_lines_energy=xrf_lines_energy
		self.pxrf=pxrf
		self.sxrf=sxrf

class Constant_scat:
	def __init__(self,energy_compton,mu_coh,mu_incoh,mu_photo,salpha_lamda) -> None:
		self.energy_compton= energy_compton
		self.mu_coh=mu_coh
		self.mu_incoh=mu_incoh
		self.mu_photo=mu_photo
		self.salpha_lamda=salpha_lamda
		pass

In [3]:
!pip install scipy

In [4]:
'''
;====================================================================================================
;                              X2ABUNDANCE
;
; Package for determining elemental weight percentages from XRF line fluxes
;
; Algorithm developed by P. S. Athiray (Athiray et al. 2015)
; Codes in IDL written by Netra S Pillai
; Codes for XSPEC localmodel developed by Ashish Jacob Sam and Netra S Pillai
;
; Developed at Space Astronomy Group, U.R.Rao Satellite Centre, Indian Space Research Organisation
;
;====================================================================================================

This file contains the function get_constants_xrf that interpolates the cross-sections from the database to the input energy axis and also takes into account inter-element effects
'''

# from common_modules import *
from scipy.interpolate import interp1d

def get_constants_xrf(energy:list,at_no:list,weight:list,xrf_lines:Xrf_Lines) -> Const_Xrf:
    #; Function to compute the different cross sections necessary for computing XRF lines

	#Original: weight = weight/total(weight) ; To confirm that weight fractions are taken
    totalweight = np.sum(weight)
    weight = weight/totalweight
    no_elements = n_elements(at_no)
    n_ebins = n_elements(energy)
    
    #; Identify the number of lines for which xrf computation is done - just by checking array sizes of xrf_lines
    tmp2 = xrf_lines.edgeenergy
    n_lines = np.shape(tmp2)[1]
    
    #; Computing total attenuation of sample at characteristic line energies
    musampletotal_echarline = dblarr(no_elements,n_lines)
    for i in range(0, no_elements):
        for j in range(0, n_lines):
            line_energy = xrf_lines.lineenergy[i,j]
            rad_rate = xrf_lines.radrate[i,j]
            if ((line_energy > 0) and (rad_rate > 0)):
                for k in range(0,no_elements):
                    tmp3 = np.where(xrf_lines.energy_nist[k,:] != 0)
                    x_interp = (xrf_lines.energy_nist[k,tmp3])[0,:]
                    y_interp = (xrf_lines.totalcs_nist[k,tmp3])[0,:]
                    func_interp = interp1d(x_interp, y_interp, fill_value='extrapolate')
                    muelement_echarline = func_interp(line_energy)
                    musampletotal_echarline[i,j] = musampletotal_echarline[i,j] + weight[k]*muelement_echarline
                    
    #; Computing total attenuation of sample for incident energy, but only if incident energy is greater than the edge corresponding for transition
    musampletotal_eincident = dblarr(no_elements,n_lines,n_ebins)
    for i in range(0,no_elements):
        for j in range(0,n_lines):
            line_energy = xrf_lines.lineenergy[i,j]
            rad_rate = xrf_lines.radrate[i,j]
            edge_energy = xrf_lines.edgeenergy[i,j]
            if ((line_energy > 0) and (rad_rate > 0)):#; This is to ensure that only proper XRF lines are taken
                for k in range(0,no_elements):
                    tmp3 = np.where(xrf_lines.energy_nist[k,:] != 0)
                    x_interp = (xrf_lines.energy_nist[k,tmp3])[0,:]
                    y_interp = (xrf_lines.totalcs_nist[k,tmp3])[0,:]
                    func_interp = interp1d(x_interp, y_interp, fill_value='extrapolate')
                    muelement_eincident = func_interp(energy)
                    musampletotal_eincident[i,j,:] = musampletotal_eincident[i,j,:] + weight[k]*muelement_eincident 
                tmp4 = np.where(energy < edge_energy)
                if (n_elements(tmp4) != 0):
                    musampletotal_eincident[i,j,tmp4] = 0.0
    
    #; Computing photoelectric attenuation of element for incident energy, but only if incident energy is greater than the edge corresponding to the transition
    muelementphoto_eincident = dblarr(no_elements,n_lines,n_ebins)
    for i in range(0,no_elements):
        for j in range(0,n_lines):
            line_energy = xrf_lines.lineenergy[i,j]
            rad_rate = xrf_lines.radrate[i,j]
            edge_energy = xrf_lines.edgeenergy[i,j]
            if ((line_energy > 0) and (rad_rate > 0)):#; This is to ensure that only proper XRF lines are taken
                tmp3 = np.where(xrf_lines.energy_nist[i,:] != 0)
                x_interp = (xrf_lines.energy_nist[i,tmp3])[0,:]
                y_interp = (xrf_lines.photoncs_nist[i,tmp3])[0,:]
                func_interp = interp1d(x_interp, y_interp, fill_value='extrapolate')
                muelement_eincident = func_interp(energy)
                muelementphoto_eincident[i,j,:] = muelement_eincident
                tmp4 = np.where(energy < edge_energy)
                if (n_elements(tmp4) != 0):
                    muelementphoto_eincident[i,j,tmp4] = 0.0 #; Setting values less than the edge energy as 0
                    
    #; Creating a structure to return the data
    return Const_Xrf(musampletotal_echarline, musampletotal_eincident,muelementphoto_eincident)



In [6]:
'''
;====================================================================================================
;                              X2ABUNDANCE
;
; Package for determining elemental weight percentages from XRF line fluxes
;
; Algorithm developed by P. S. Athiray (Athiray et al. 2015)
; Codes in IDL written by Netra S Pillai
; Codes for XSPEC localmodel developed by Ashish Jacob Sam and Netra S Pillai
;
; Developed at Space Astronomy Group, U.R.Rao Satellite Centre, Indian Space Research Organisation
;
;====================================================================================================

This file contains the function get_xrf_lines which derives various constants (cross sections, fluorescent yields, jump factors etc) for the elements of interest

'''
# from common_modules import *
import xraylib
import os

def get_xrf_lines(at_no, k_shell, k_lines, l1_shell, l1_lines, l2_shell, l2_lines, l3_shell, l3_lines) -> Xrf_Lines:
    no_elements = n_elements(at_no)
    edgeenergy = dblarr(no_elements, 5)
    fluoryield = dblarr(no_elements, 5)
    jumpfactor = dblarr(no_elements, 5)
    radrate = dblarr(no_elements, 5)
    lineenergy = dblarr(no_elements, 5)
    
    energy_nist = dblarr(no_elements, 100)
    photoncs_nist = dblarr(no_elements, 100)
    totalcs_nist = dblarr(no_elements, 100)
    elename_string = strarr(no_elements)

    __file__ = "/mnt/c/Users/r9307/interIIT/ch2_class_pds_release_38_20240927/cla/miscellaneous/ch2_class_x2abund_lmodel_v1.0/X2ABUND_LMODEL_V1/data_constants/kalpha_be_density_kbeta.txt"

    (atomic_number_list, kalpha_list, ele_list, be_list, density_list, kbeta_list) = readcol(__file__, format='I,F,A,F,F,F')

    fullpath = os.path.abspath(__file__)
    script_path, filename = os.path.split(fullpath)
    
    for i in range(0, no_elements):
        #; Getting the NIST cross-sections for all the elements
        tmp1 = np.where(atomic_number_list == at_no[i])
        elename_string[i] = ele_list[tmp1]
        
        filename = script_path + '/data_constants/ffast/ffast_'+str(int(at_no[i])).strip()+'_'+(ele_list[tmp1])[0]+'.txt'#; Getting the attenuation coefficients from FFAST database
        (column1, column2, column3, column4, column5, column6, column7, column8) = readcol(filename, format = 'D,F,F,F,F,F,F,F')
        
        n = n_elements(column1)
        energy_nist[i, 0:n] = column1
        photoncs_nist[i, 0:n] = column4
        totalcs_nist[i, 0:n] = column6
        
        #; Getting the edge energy for each shell
        edgeenergy[i, 0:2] = xraylib.EdgeEnergy(at_no[i], k_shell)#; edge energy is defined for shells so it will be same for kalpha and kbeta
        edgeenergy[i, 2] = xraylib.EdgeEnergy(at_no[i], l1_shell)
        edgeenergy[i, 3] = xraylib.EdgeEnergy(at_no[i], l2_shell)
        edgeenergy[i, 4] = xraylib.EdgeEnergy(at_no[i], l3_shell)
        
        #; Getting the fluorescent yields for each shell
        fluoryield[i, 0:2] = xraylib.FluorYield(at_no[i], k_shell)#; fluorescent yield is defined for shells so it will be same for kalpha and kbeta
        try:
            fluoryield[i, 2] = xraylib.FluorYield(at_no[i], l1_shell)
        except:
            fluoryield[i, 2] = 0.0
        try:
            fluoryield[i, 3] = xraylib.FluorYield(at_no[i], l2_shell)
        except:
            fluoryield[i, 3] = 0.0
        try:
            fluoryield[i, 4] = xraylib.FluorYield(at_no[i], l3_shell)
        except:
            fluoryield[i, 4] = 0.0
       
        #; Getting the jump factors for each shell
        jumpfactor[i, 0:2] = xraylib.JumpFactor(at_no[i], k_shell)#; jump factor is defined for shells so it will be same for kalpha and kbeta
        try:
            jumpfactor[i, 2] = xraylib.JumpFactor(at_no[i], l1_shell)
        except:
            jumpfactor[i, 2] = 0.0
        try:
            jumpfactor[i, 3] = xraylib.JumpFactor(at_no[i], l2_shell)
        except:
            jumpfactor[i, 3] = 0.0
        try:
            jumpfactor[i, 4] = xraylib.JumpFactor(at_no[i], l3_shell)
        except:
            jumpfactor[i, 4] = 0.0
        
        #; Getting the radiative rates and energy for kbeta
        kbeta_lines = k_lines[3:8]
        kbeta_lines_length = n_elements(kbeta_lines)
        radrate_kbeta = dblarr(kbeta_lines_length)
        lineenergy_kbeta = dblarr(kbeta_lines_length)
        for j in range(0, kbeta_lines_length):
            try:
                radrate_kbeta[j] = xraylib.RadRate(at_no[i], kbeta_lines[j])
                lineenergy_kbeta[j] = xraylib.LineEnergy(at_no[i], kbeta_lines[j])
            except:
                radrate_kbeta[j] = 0.0
                lineenergy_kbeta[j] = 0.0

        allowed_lines_index_kbeta = np.where(radrate_kbeta > 0)
        if (n_elements(allowed_lines_index_kbeta) != 0):# then begin
            	lineenergy[i, 0] = total(radrate_kbeta[allowed_lines_index_kbeta]*lineenergy_kbeta[allowed_lines_index_kbeta])/total(radrate_kbeta[allowed_lines_index_kbeta])#; weighted average of kbeta energies
            	radrate[i, 0] = total(radrate_kbeta[allowed_lines_index_kbeta])
		# If no kbeta line is possible then the energy and the radrate will be set as 0.

        #; Getting the radiative rates and energy for kalpha
        kalpha_lines = k_lines[0:3]
        radrate_kalpha = dblarr(n_elements(kalpha_lines))
        lineenergy_kalpha = dblarr(n_elements(kalpha_lines))
        for j in range(0, n_elements(kalpha_lines)):#-1 do begin
            try:
                radrate_kalpha[j] = xraylib.RadRate(at_no[i], kalpha_lines[j])
                lineenergy_kalpha[j] = xraylib.LineEnergy(at_no[i], kalpha_lines[j])
            except:
                radrate_kalpha[j] = 0.0
                lineenergy_kalpha[j] = 0.0

        allowed_lines_index_kalpha = np.where(radrate_kalpha > 0)
        if (n_elements(allowed_lines_index_kalpha) != 0):# then begin
            lineenergy[i, 1] = total(radrate_kalpha[allowed_lines_index_kalpha]*lineenergy_kalpha[allowed_lines_index_kalpha])/total(radrate_kalpha[allowed_lines_index_kalpha])#; weighted average of kalpha energies
            radrate[i, 1] = total(radrate_kalpha[allowed_lines_index_kalpha])

        #; Getting the radiative rates and energy for l1lines
        radrate_l1 = dblarr(n_elements(l1_lines))
        lineenergy_l1 = dblarr(n_elements(l1_lines))
        for j in range(0, n_elements(l1_lines)):#-1 do begin
            try:
                radrate_l1[j] = xraylib.RadRate(at_no[i], l1_lines[j])
                lineenergy_l1[j] = xraylib.LineEnergy(at_no[i], l1_lines[j])
            except:
                radrate_l1[j] = 0.0
                lineenergy_l1[j] = 0.0

        allowed_lines_index_l1 = np.where(radrate_l1 > 0)
        if (n_elements(allowed_lines_index_l1) != 0):# then begin
            lineenergy[i, 2] = total(radrate_l1[allowed_lines_index_l1]*lineenergy_l1[allowed_lines_index_l1])/total(radrate_l1[allowed_lines_index_l1])#; weighted average of l1 energies
            radrate[i, 2] = total(radrate_l1[allowed_lines_index_l1])
		# If no l1 line is possible then the energy and the radrate will be set as 0.

        #; Getting the radiative rates and energy for l2lines
        radrate_l2 = dblarr(n_elements(l2_lines))
        lineenergy_l2 = dblarr(n_elements(l2_lines))
        for j in range(0, n_elements(l2_lines)):#-1 do begin
            try:
                radrate_l2[j] = xraylib.RadRate(at_no[i], l2_lines[j])
                lineenergy_l2[j] = xraylib.LineEnergy(at_no[i], l2_lines[j])
            except:
                radrate_l2[j] = 0.0
                lineenergy_l2[j] = 0.0

        allowed_lines_index_l2 = np.where(radrate_l2 > 0)
        if (n_elements(allowed_lines_index_l2) != 0):# then begin
            lineenergy[i, 3] = total(radrate_l2[allowed_lines_index_l2]*lineenergy_l2[allowed_lines_index_l2])/total(radrate_l2[allowed_lines_index_l2])#; weighted average of l2 energies
            radrate[i, 3] = total(radrate_l2[allowed_lines_index_l2])
		# If no l2 line is possible then the energy and the radrate will be set as 0.

        #; Getting the radiative rates and energy for l3lines
        radrate_l3 = dblarr(n_elements(l3_lines))
        lineenergy_l3 = dblarr(n_elements(l3_lines))
        for j in range(0, n_elements(l3_lines)):#-1 do begin
            try:
                radrate_l3[j] = xraylib.RadRate(at_no[i], l3_lines[j])
                lineenergy_l3[j] = xraylib.LineEnergy(at_no[i], l3_lines[j])
            except:
                radrate_l3[j] = 0.0
                lineenergy_l3[j] = 0.0

        allowed_lines_index_l3 = np.where(radrate_l3 > 0)
        if (n_elements(allowed_lines_index_l3[0]) != 0):# then begin
            lineenergy[i, 4] = total(radrate_l3[allowed_lines_index_l3]*lineenergy_l3[allowed_lines_index_l3])/total(radrate_l3[allowed_lines_index_l3])#; weighted average of l3 energies
            radrate[i, 4] = total(radrate_l3[allowed_lines_index_l3])
		# If no l3 line is possible then the energy and the radrate will be set as 0.

    #; Creating a structure to return the data
    return Xrf_Lines(edgeenergy,fluoryield,jumpfactor,radrate,lineenergy,energy_nist,photoncs_nist,totalcs_nist,elename_string)

In [8]:
'''
;====================================================================================================
;                              X2ABUNDANCE
;
; Package for determining elemental weight percentages from XRF line fluxes
;
; Algorithm developed by P. S. Athiray (Athiray et al. 2015)
; Codes in IDL written by Netra S Pillai
; Codes for XSPEC localmodel developed by Ashish Jacob Sam and Netra S Pillai
;
; Developed at Space Astronomy Group, U.R.Rao Satellite Centre, Indian Space Research Organisation
;
;====================================================================================================

This file contains the function xrf_comp that computes the actual XRF line intensity from each element.
'''

# from common_modules import *
import numpy as np
from scipy.interpolate import interp1d

def xrf_comp(energy: list, counts, i_angle, e_angle, at_no: list, weight: list, xrf_lines:Xrf_Lines, const_xrf:Const_Xrf) -> Xrf_Struc:
    no_elements = n_elements(at_no)
    totalweight = np.sum(weight)
    # ;Just to ensure that weight is converted to weight fractions
    weight = weight/totalweight
    tmp1 = xrf_lines.edgeenergy
    # ; To get the number of lines for each element which is to be computed
    n_lines = np.shape(tmp1)[1]
    n_ebins = n_elements(energy)
    binsize = energy[1] - energy[0]
    
    primary_xrf = dblarr(no_elements, n_lines)
    secondary_xrf = dblarr(no_elements, n_lines)
    secondary_xrf_linewise = dblarr(no_elements, n_lines, no_elements, n_lines)
    
    for i in range(0, no_elements):
        for j in range(0, n_lines):
            fluoryield = xrf_lines.fluoryield[i, j]
            radrate = xrf_lines.radrate[i, j]
            lineenergy = xrf_lines.lineenergy[i, j]
            
            #; Computing the probability coming from the jump ratios
            element_jumpfactor = xrf_lines.jumpfactor[i, :]
            element_edgeenergy = xrf_lines.edgeenergy[i, :]
            ratio_jump = dblarr(n_ebins)
            if (j <= 1):#; Jump ratio probability for K-transitions
                tmp2 = np.where(energy >= element_edgeenergy[j])
                ratio_jump[tmp2] = 1.0 - 1.0/element_jumpfactor[j]
            else:#; Jump ratio probability for L and higher transitions
                tmp3 = np.where(energy > element_edgeenergy[1])
                ratio_jump[tmp3] = 1.0/PRODUCT(element_jumpfactor[1:j])*(1.-1.0/element_jumpfactor[j])
                for k in range(2, j+1):
                    tmp4 = np.where((energy < element_edgeenergy[k-1]) & (energy > element_edgeenergy[k]))
                    if (n_elements(tmp4)!=0):
                        if (k != j):
                            ratio_jump[tmp4] = 1.0/PRODUCT(element_jumpfactor[k:j])*(1.-1.0/element_jumpfactor[j])
                        else:
                            ratio_jump[tmp4] = (1.-1.0/element_jumpfactor[j])
                            
            if((lineenergy > 0) and (radrate > 0)):
                
                #; Computing primary xrf
                musample_eincident = const_xrf.musampletotal_eincident[i, j, :]
                musample_echarline = const_xrf.musampletotal_echarline[i, j]
                muelement_eincident = const_xrf.muelementphoto_eincident[i, j, :]
                pxrf_denom = musample_eincident*(1.0/np.sin(i_angle * np.pi/180)) + musample_echarline*(1.0/np.sin(e_angle * np.pi/180))
                pxrf_Q = weight[i]*muelement_eincident*fluoryield*radrate*ratio_jump
                primary_xrf[i, j] = (1.0/np.sin(i_angle * np.pi/180))*total((pxrf_Q*counts*binsize)/(pxrf_denom))
                
                #; Computing secondary xrf(i.e secondary enhancement in other lines due to this line)
                secondaries_index_2D = np.where(xrf_lines.edgeenergy < lineenergy)
                secondaries_index_2D = np.array(secondaries_index_2D)
                n_secondaries = (np.shape(secondaries_index_2D))[1]
                for k in range(0, n_secondaries):
                    i_secondary = secondaries_index_2D[0,k]
                    j_secondary = secondaries_index_2D[1,k]
                    
                    fluoryield_secondary = xrf_lines.fluoryield[i_secondary, j_secondary]
                    radrate_secondary = xrf_lines.radrate[i_secondary, j_secondary]
                    lineenergy_secondary = xrf_lines.lineenergy[i_secondary, j_secondary]
                    
                    #; Computing the probability coming from the jump ratios for secondaries
                    element_jumpfactor_secondary = xrf_lines.jumpfactor[i_secondary, :]
                    if (j_secondary <= 1):#; Jump ratio probability for K-transitions
                        ratio_jump_secondary = 1.0 - 1.0/element_jumpfactor[j_secondary]
                    else:#; Jump ratio probability for L and higher transitions
                        ratio_jump_secondary = 1.0/PRODUCT(element_jumpfactor_secondary[1:j_secondary])*(1.-1.0/element_jumpfactor_secondary[j_secondary])
                        
                    if((lineenergy_secondary > 0) and (radrate_secondary > 0)):
                        musample_echarline_secondary = const_xrf.musampletotal_echarline[i_secondary, j_secondary]
                        muelement_eincident_secondary = const_xrf.muelementphoto_eincident[i_secondary, j_secondary, :]
                        
                        x_interp = energy
                        y_interp = muelement_eincident_secondary
                        func_interp = interp1d(x_interp, y_interp, fill_value='extrapolate')
                        muelement_pline_secondary = func_interp(lineenergy)
                        
                        L = 0.5*((((np.sin(i_angle * np.pi/180))/(musample_eincident))*np.log(1+(musample_eincident)/(np.sin(i_angle * np.pi/180)*musample_echarline))) + (((np.sin(e_angle * np.pi/180))/(musample_echarline_secondary))*np.log(1+(musample_echarline_secondary)/(np.sin(e_angle * np.pi/180)*musample_echarline))))
                        zero_index = np.where(musample_eincident == 0)#; This is to avoid places of division with 0
                        if (n_elements(zero_index) != 0):
                            L[zero_index] = 0
                            
                        sxrf_denom = musample_eincident*(1.0/np.sin(i_angle * np.pi/180)) + musample_echarline_secondary*(1.0/np.sin(e_angle * np.pi/180))
                        sxrf_Q = weight[i_secondary]*muelement_pline_secondary*fluoryield_secondary*radrate_secondary*ratio_jump_secondary
                        
                        secondary_xrf_linewise[i, j, i_secondary, j_secondary] = (1.0/np.sin(i_angle * np.pi/180))*total((counts*pxrf_Q*sxrf_Q*L*binsize)/(sxrf_denom))
                        if (secondary_xrf_linewise[i, j, i_secondary, j_secondary] > 0):
                            secondary_xrf[i_secondary, j_secondary] = secondary_xrf[i_secondary, j_secondary] + secondary_xrf_linewise[i, j, i_secondary, j_secondary]
                            
    total_xrf = primary_xrf + secondary_xrf
    return Xrf_Struc(primary_xrf, secondary_xrf, total_xrf)

In [9]:
"""
;====================================================================================================
;                              X2ABUNDANCE
;
; Package for determining elemental weight percentages from XRF line fluxes
;
; Algorithm developed by P. S. Athiray (Athiray et al. 2015)
; Codes in IDL written by Netra S Pillai
; Codes for XSPEC localmodel developed by Ashish Jacob Sam and Netra S Pillai
;
; Developed at Space Astronomy Group, U.R.Rao Satellite Centre, Indian Space Research Organisation
;
;====================================================================================================

This is the local model defined for fitting CLASS data using PyXspec

"""
# Importing necessary modules
import numpy as np
from xspec import *
import xraylib
# from common_modules import *
# from get_xrf_lines_V1 import get_xrf_lines
# from get_constants_xrf_new_V2 import get_constants_xrf
# from xrf_comp_new_V2 import xrf_comp

# Getting the static parameters for the local model
static_parameter_file = "static_par_localmodel.txt"
fid = open(static_parameter_file,"r")
finfo_full = fid.read()
finfo_split = finfo_full.split('\n')
solar_file = finfo_split[0]
solar_zenith_angle = float(finfo_split[1])
emiss_angle = float(finfo_split[2])
altitude = float(finfo_split[3])
exposure = float(finfo_split[4])

# Defining the model function
def xrf_localmodel(energy, parameters, flux):
    
    # Defining proper energy axis
    energy_mid = np.zeros(np.size(energy)-1)
    for i in np.arange(np.size(energy)-1):
        energy_mid[i] = 0.5*(energy[i+1] + energy[i])
        
    # Defining some input parameters required for x2abund xrf computation modules
    at_no = np.array([26,22,20,14,13,12,11,8])
    
    weight = list(parameters)
    
    i_angle = 90.0 - solar_zenith_angle
    e_angle = 90.0 - emiss_angle
    (energy_solar,tmp1_solar,counts_solar) = readcol(solar_file,format='F,F,F')
    
    # Computing the XRF line intensities
    k_lines = np.array([xraylib.KL1_LINE, xraylib.KL2_LINE, xraylib.KL3_LINE, xraylib.KM1_LINE, xraylib.KM2_LINE, xraylib.KM3_LINE, xraylib.KM4_LINE, xraylib.KM5_LINE])
    l1_lines = np.array([xraylib.L1L2_LINE, xraylib.L1L3_LINE, xraylib.L1M1_LINE, xraylib.L1M2_LINE, xraylib.L1M3_LINE, xraylib.L1M4_LINE, xraylib.L1M5_LINE, xraylib.L1N1_LINE, xraylib.L1N2_LINE, xraylib.L1N3_LINE, xraylib.L1N4_LINE, xraylib.L1N5_LINE, xraylib.L1N6_LINE, xraylib.L1N7_LINE])
    l2_lines = np.array([xraylib.L2L3_LINE, xraylib.L2M1_LINE, xraylib.L2M2_LINE, xraylib.L2M3_LINE, xraylib.L2M4_LINE, xraylib.L2M5_LINE, xraylib.L2N1_LINE, xraylib.L2N2_LINE, xraylib.L2N3_LINE, xraylib.L2N4_LINE, xraylib.L2N5_LINE, xraylib.L2N6_LINE, xraylib.L2N7_LINE])
    l3_lines = [xraylib.L3M1_LINE, xraylib.L3M2_LINE, xraylib.L3M3_LINE, xraylib.L3M4_LINE, xraylib.L3M5_LINE, xraylib.L3N1_LINE,xraylib.L3N2_LINE, xraylib.L3N3_LINE, xraylib.L3N4_LINE, xraylib.L3N5_LINE, xraylib.L3N6_LINE, xraylib.L3N7_LINE]
    xrf_lines = get_xrf_lines(at_no, xraylib.K_SHELL, k_lines, xraylib.L1_SHELL, l1_lines, xraylib.L2_SHELL, l2_lines, xraylib.L3_SHELL, l3_lines)
    const_xrf = get_constants_xrf(energy_solar, at_no, weight, xrf_lines)
    xrf_struc = xrf_comp(energy_solar,counts_solar,i_angle,e_angle,at_no,weight,xrf_lines,const_xrf)
    
    # Generating XRF spectrum
    bin_size = energy[1] - energy[0]
    ebin_left = energy_mid - 0.5*bin_size
    ebin_right = energy_mid + 0.5*bin_size
    
    no_elements = (np.shape(xrf_lines.lineenergy))[0]
    n_lines = (np.shape(xrf_lines.lineenergy))[1]
    n_ebins = np.size(energy_mid)
    
    spectrum_xrf = dblarr(n_ebins)
    for i in range(0, no_elements):
        for j in range(0, n_lines):
            line_energy = xrf_lines.lineenergy[i,j]
            bin_index = np.where((ebin_left <= line_energy) & (ebin_right >= line_energy))
            spectrum_xrf[bin_index] = spectrum_xrf[bin_index] + xrf_struc.total_xrf[i,j]
            
    # Defining the flux array required for XSPEC
    scaling_factor = (12.5*1e4*12.5*(round(exposure/8.0)+1)*1e4)/(exposure*4*np.pi*(altitude*1e4)**2)
    spectrum_xrf_scaled = scaling_factor*spectrum_xrf
    for i in range(0, n_ebins):
        flux[i] = spectrum_xrf_scaled[i]
        
# Specifying parameter information
xrf_localmodel_ParInfo = ("Wt_Fe \"\" 5 1 1 20 20 1e-2","Wt_Ti \"\" 1 1e-6 1e-6 20 20 1e-2","Wt_Ca \"\" 9 5 5 20 20 1e-2","Wt_Si \"\" 21 15 15 35 35 1e-2","Wt_Al \"\" 14 5 5 20 20 1e-2","Wt_Mg \"\" 5 1e-6 1e-6 20 20 1e-2","Wt_Na \"\" 0.5 1e-6 1e-6 5 5 1e-2","Wt_O \"\" 45 30 30 60 60 1e-2")

# Creating the local model in PyXspec
AllModels.addPyMod(xrf_localmodel, xrf_localmodel_ParInfo, 'add')
    

In [2]:
import os
from xspec import *

In [10]:
# Path to the directory containing the FITS files
fits_directory = "/mnt/c/Users/r9307/interIIT/ch2_class_pds_release_38_20240927/cla/calibration/pradan.issdc.gov.in/ch2/protected/downloadData/POST_OD/isda_archive/ch2_bundle/cho_bundle/nop/cla_collection/cla/data/calibrated/2024/02/01"

In [11]:
if not os.path.exists(fits_directory):
    print("Directory not found:", fits_directory)  # Checkpoint 2
else:
    print("Directory exists. Proceeding...")

Directory exists. Proceeding...


In [ ]:


# Path to the define_xrf_localmodel.py script
# local_model_script = "/path/to/define_xrf_localmodel.py"

# Load the local XRF model
# exec(open(local_model_script).read())
print("Starting loop...")
# Loop over each FITS file in the directory
for fits_file in os.listdir(fits_directory):
    if fits_file.endswith(".fits"):
        print(f"processing{fits_file}")
        fits_path = os.path.join(fits_directory, fits_file)

        calibration_path = "/mnt/c/Users/r9307/interIIT/ch2_class_pds_release_38_20240927/cla/calibration"
        respfile = calibration_path + 'class_rmf_v1.rmf'
        arrffile = calibration_path + 'class_arf_v1.arf'
        backFile = calibration_path + 'background_allevents.fits'

        print("loading Spectrum...")
        # Load the FITS file in XSPEC
        Spectrum(dataFile=fits_path, respFile=respfile, arfFile=arrffile, backFile=backFile)

        print("loading model...")
        
        # Apply the model to the loaded spectrum
        model = Model("xrf_localmodel")
        
        # Set model parameters here if needed, e.g., model(1).values = <some_value>
        print("fitting model...")
        # Fit the model to the spectrum
        Fit.perform()
        
        # Output results, e.g., saving the results or displaying the best-fit parameters
        print(f"Results for {fits_file}:")
        Fit.show()
        AllModels.show()

        # Optional: Save output to a file
        with open(f"{fits_file}_fit_results.txt", "w") as output_file:
            output_file.write(f"Results for {fits_file}:\n")
            output_file.write(Fit.show())
            output_file.write(AllModels.show())
